In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Conv2D, MaxPool2D, GlobalAveragePooling2D, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers

In [2]:
train_path = 'Waste-Net'
valid_path = 'Waste-Net'

In [3]:
train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.inception_v3.preprocess_input,
    shear_range=0.2,
    zoom_range=0.2,
    validation_split=0.1)
valid_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.inception_v3.preprocess_input,
    validation_split=0.1)

In [4]:
class_subset=['cardboard', 'ewaste', 'glass', 'metal','organic', 'paper', 'plastic']
train_batches= train_generator.flow_from_directory(directory=train_path,
                                               target_size=(224, 224),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='training',
                                               batch_size=32, 
                                               shuffle=True,
                                               seed=42)
valid_batches= valid_generator.flow_from_directory(directory=valid_path,
                                               target_size=(224, 224),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='validation',
                                               batch_size=32,
                                               shuffle=True,
                                               seed=42)

Found 2857 images belonging to 7 classes.
Found 314 images belonging to 7 classes.


In [13]:
IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
base_model = InceptionV3(input_shape=IMG_SHAPE, 
                              include_top=False,
                              weights = 'imagenet')

In [14]:
base_model.trainable=False

In [15]:
last_layer = base_model.get_layer('mixed8')
last_output = last_layer.output

In [8]:
x = layers.Flatten()(last_output)
x = layers.Dropout(0.2)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(7, activation='softmax')(x)

In [10]:
model = Model(base_model.input, x)

In [11]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [12]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_details = model.fit(x=train_batches, validation_data=valid_batches, epochs=10, verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 90 steps, validate for 10 steps
Epoch 1/10
90/90 [==============================] - 831s 9s/step - loss: 0.7580 - accuracy: 0.7242 - val_loss: 0.4699 - val_accuracy: 0.8217
Epoch 2/10
90/90 [==============================] - 781s 9s/step - loss: 0.1511 - accuracy: 0.9503 - val_loss: 0.4047 - val_accuracy: 0.8885
Epoch 3/10
90/90 [==============================] - 793s 9s/step - loss: 0.0507 - accuracy: 0.9839 - val_loss: 0.3085 - val_accuracy: 0.9076
Epoch 4/10
72/90 [=======================>......] - ETA: 2:34 - loss: 0.0202 - accuracy: 0.9952

Model 2 - Inception V3

In [5]:
IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
base_model2 = InceptionV3(input_shape=IMG_SHAPE, 
                              include_top=False,
                              weights = 'imagenet')

In [6]:
base_model2.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [7]:
model2 = Sequential()
base_model2.trainable=False
model2.add(base_model2)
model2.add(GlobalAveragePooling2D()) 
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 5, 5, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
Total params: 21,802,784
Trainable params: 0
Non-trainable params: 21,802,784
_________________________________________________________________


In [8]:
model2.add(Dense(units=1024, activation='relu'))  
model2.add(BatchNormalization())                 
model2.add(Dropout(0.2))                         

model2.add(Dense(units=512, activation='relu')) 
model2.add(BatchNormalization())               
model2.add(Dropout(0.2))                        

model2.add(Dense(units=7, activation='softmax'))
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 5, 5, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
batch_normalization_94 (Batc (None, 1024)              4096      
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
batch_normalization_95 (Batc (None, 512)               2

In [9]:
model2.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model_details2 = model2.fit(x=train_batches, validation_data=valid_batches, epochs=10, verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 90 steps, validate for 10 steps
Epoch 1/10
90/90 [==============================] - 459s 5s/step - loss: 1.5082 - accuracy: 0.4900 - val_loss: 0.9634 - val_accuracy: 0.6529
Epoch 2/10
90/90 [==============================] - 429s 5s/step - loss: 0.8284 - accuracy: 0.7245 - val_loss: 0.8641 - val_accuracy: 0.6879
Epoch 3/10
90/90 [==============================] - 429s 5s/step - loss: 0.6084 - accuracy: 0.7893 - val_loss: 0.9372 - val_accuracy: 0.7070
Epoch 4/10
90/90 [==============================] - 430s 5s/step - loss: 0.5027 - accuracy: 0.8299 - val_loss: 1.0846 - val_accuracy: 0.7102
Epoch 5/10
90/90 [==============================] - 428s 5s/step - loss: 0.4193 - accuracy: 0.8530 - val_loss: 1.1440 - val_accuracy: 0.6911
Epoch 6/10
90/90 [==============================] - 429s 5s/step - loss: 0.3657 - accuracy: 0.8645 - val_loss: 0.9587 - val_accuracy: 0.7293
Epoch 7/10
90/90 [==============================] - 428s 5s/st

In [12]:
tr_loss_2=model_details2.history['loss']
val_loss_2=model_details2.history['val_loss']
tr_acc_2=model_details2.history['accuracy']
val_acc_2=model_details2.history['val_accuracy']

In [16]:
IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
base_model3 = InceptionV3(input_shape=IMG_SHAPE, 
                              include_top=False,
                              weights = 'imagenet')

In [17]:
base_model3.trainable=False
for layer in base_model3.layers:
    print(layer.trainable)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [20]:
from tensorflow.keras.layers import AveragePooling2D
headModel = base_model3.output
headModel = AveragePooling2D(pool_size=(5, 5))(headModel)

In [22]:
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.2)(headModel)
headModel = Dense(7, activation="softmax")(headModel)

In [23]:
model = Model(inputs=base_model3.input, outputs=headModel)

In [24]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, 111, 111, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_190 (BatchN (None, 111, 111, 32) 96          conv2d_188[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 111, 111, 32) 0           batch_normalization_190[0][0]    
______________________________________________________________________________________________

In [25]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
model_details3 = model.fit(x=train_batches, validation_data=valid_batches, epochs=10, verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 90 steps, validate for 10 steps
Epoch 1/10
90/90 [==============================] - 343s 4s/step - loss: 1.4487 - accuracy: 0.4760 - val_loss: 0.9664 - val_accuracy: 0.6592
Epoch 2/10
90/90 [==============================] - 347s 4s/step - loss: 0.9428 - accuracy: 0.6867 - val_loss: 0.8226 - val_accuracy: 0.7325
Epoch 3/10
90/90 [==============================] - 334s 4s/step - loss: 0.7506 - accuracy: 0.7462 - val_loss: 0.8341 - val_accuracy: 0.7197
Epoch 4/10
90/90 [==============================] - 330s 4s/step - loss: 0.6551 - accuracy: 0.7798 - val_loss: 0.8310 - val_accuracy: 0.7293
Epoch 5/10
90/90 [==============================] - 328s 4s/step - loss: 0.5987 - accuracy: 0.8008 - val_loss: 0.8341 - val_accuracy: 0.7420
Epoch 6/10
90/90 [==============================] - 326s 4s/step - loss: 0.5539 - accuracy: 0.8173 - val_loss: 0.7487 - val_accuracy: 0.7707
Epoch 7/10
90/90 [==============================] - 327s 4s/st

In [31]:
base_model3.trainable=True
model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [53]:
tr_loss_3=model_details3.history['loss']
val_loss_3=model_details3.history['val_loss']
tr_acc_3=model_details3.history['accuracy']
val_acc_3=model_details3.history['val_accuracy']

In [33]:
model_details4 = model.fit(x=train_batches, validation_data=valid_batches, epochs=5, verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 90 steps, validate for 10 steps
Epoch 1/5
90/90 [==============================] - 905s 10s/step - loss: 0.3707 - accuracy: 0.8778 - val_loss: 0.5847 - val_accuracy: 0.8185
Epoch 2/5
90/90 [==============================] - 899s 10s/step - loss: 0.2035 - accuracy: 0.9461 - val_loss: 0.5483 - val_accuracy: 0.8248
Epoch 3/5
90/90 [==============================] - 860s 10s/step - loss: 0.1412 - accuracy: 0.9664 - val_loss: 0.5209 - val_accuracy: 0.8439
Epoch 4/5
90/90 [==============================] - 859s 10s/step - loss: 0.0923 - accuracy: 0.9853 - val_loss: 0.4836 - val_accuracy: 0.8535
Epoch 5/5
90/90 [==============================] - 899s 10s/step - loss: 0.0651 - accuracy: 0.9926 - val_loss: 0.4760 - val_accuracy: 0.8631


In [34]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [38]:
img1 = load_img('test4.jpg', target_size=(224,224))
img_array = tf.keras.preprocessing.image.img_to_array(img1)
img_array2=tf.keras.applications.inception_v3.preprocess_input(img_array)
img_array2 = tf.expand_dims(img_array2, 0)
predictions = model2.predict(img_array2)
predicted_class = class_subset[np.argmax(predictions[0])]
confidence = round(100 * (np.max(predictions[0])), 2)
print(confidence,predicted_class)

99.93 plastic


In [52]:
tr_loss_4=model_details4.history['loss']
val_loss_4=model_details4.history['val_loss']
tr_acc_4=model_details4.history['accuracy']
val_acc_4=model_details4.history['val_accuracy']
print(tr_acc_4)

[0.8778439, 0.9460973, 0.9663983, 0.9852993, 0.9926496]


In [41]:
model.save('inception_1_f.h5')

In [42]:
model.save("inception_f_1")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: inception_f_1\assets


In [55]:
print(tr_loss_3)
print(tr_acc_3)
print(val_loss_3)
print(val_acc_3)

[1.447855031319252, 0.9413454274760704, 0.7510246153598822, 0.6555855135672318, 0.5976197224920097, 0.5546516183480793, 0.5174627129247984, 0.47984435496148337, 0.4570953056534396, 0.4503797573416559, [0.3683144784616264, 0.2046894629505138, 0.1420729830652465, 0.09262028043534859, 0.06545167886787334], [0.3683144784616264, 0.2046894629505138, 0.1420729830652465, 0.09262028043534859, 0.06545167886787334], 0.3683144784616264, 0.2046894629505138, 0.1420729830652465, 0.09262028043534859, 0.06545167886787334]
[0.4760238, 0.6867343, 0.74623734, 0.779839, 0.80084, 0.81729084, 0.82674134, 0.8375919, 0.84564227, 0.84354216]
[0.9664466738700866, 0.8226216018199921, 0.8340984880924225, 0.8309756517410278, 0.8340901136398315, 0.748689341545105, 0.7516497135162353, 0.7725312322378158, 0.750461733341217, 0.8906608819961548]
[0.65923566, 0.7324841, 0.7197452, 0.72929937, 0.7420382, 0.77070063, 0.75159234, 0.7675159, 0.77707005, 0.7388535]


In [5]:
IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
base_model4 = InceptionV3(input_shape=IMG_SHAPE, 
                              include_top=False,
                              weights = 'imagenet')

In [6]:
base_model4.trainable=False

In [7]:
last_layer = base_model4.get_layer('mixed8')
last_output = last_layer.output

In [8]:
x=layers.AveragePooling2D(pool_size=(5, 5))(last_output)

In [9]:
x = layers.Flatten(name="flatten")(x)
x = layers.Dense(512, activation="relu")(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(7, activation="softmax")(x)

In [10]:
model4=Model(base_model4.input,x)

In [11]:
model4.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [12]:
model4.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [13]:
model_details5 = model4.fit(x=train_batches, validation_data=valid_batches, epochs=10, verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 90 steps, validate for 10 steps
Epoch 1/10
90/90 [==============================] - 419s 5s/step - loss: 1.5530 - accuracy: 0.4431 - val_loss: 1.1920 - val_accuracy: 0.5446
Epoch 2/10
90/90 [==============================] - 405s 5s/step - loss: 1.0069 - accuracy: 0.6941 - val_loss: 0.9268 - val_accuracy: 0.6561
Epoch 3/10
90/90 [==============================] - 406s 5s/step - loss: 0.7850 - accuracy: 0.7616 - val_loss: 0.7883 - val_accuracy: 0.6943
Epoch 4/10
90/90 [==============================] - 406s 5s/step - loss: 0.6585 - accuracy: 0.7914 - val_loss: 0.6575 - val_accuracy: 0.7548
Epoch 5/10
90/90 [==============================] - 405s 4s/step - loss: 0.5925 - accuracy: 0.8078 - val_loss: 0.6118 - val_accuracy: 0.7866
Epoch 6/10
90/90 [==============================] - 406s 5s/step - loss: 0.5354 - accuracy: 0.8274 - val_loss: 0.6284 - val_accuracy: 0.7548
Epoch 7/10
90/90 [==============================] - 405s 5s/st

In [19]:
model4.save('inception_2_f.h5')

In [20]:
model4.save("inception_f_2")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: inception_f_2\assets


In [21]:
base_model4.trainable=True

In [22]:
model4.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
model_details6 = model4.fit(x=train_batches, validation_data=valid_batches, epochs=4, verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 90 steps, validate for 10 steps
Epoch 1/4
90/90 [==============================] - 792s 9s/step - loss: 0.3569 - accuracy: 0.8978 - val_loss: 0.3656 - val_accuracy: 0.8567
Epoch 2/4
90/90 [==============================] - 809s 9s/step - loss: 0.2670 - accuracy: 0.9160 - val_loss: 0.3312 - val_accuracy: 0.8790
Epoch 3/4
90/90 [==============================] - 798s 9s/step - loss: 0.1947 - accuracy: 0.9506 - val_loss: 0.3065 - val_accuracy: 0.8981
Epoch 4/4
90/90 [==============================] - 800s 9s/step - loss: 0.1464 - accuracy: 0.9688 - val_loss: 0.2977 - val_accuracy: 0.8917


In [24]:
model4.save('inception_2_t.h5')

In [25]:
model4.save("inception_t_2")

INFO:tensorflow:Assets written to: inception_t_2\assets


In [26]:
model4.fit(x=train_batches, validation_data=valid_batches, epochs=2, verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 90 steps, validate for 10 steps
Epoch 1/2
90/90 [==============================] - 794s 9s/step - loss: 0.1209 - accuracy: 0.9804 - val_loss: 0.2786 - val_accuracy: 0.8885
Epoch 2/2
90/90 [==============================] - 799s 9s/step - loss: 0.0926 - accuracy: 0.9863 - val_loss: 0.2750 - val_accuracy: 0.9013


In [27]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [31]:
img1 = load_img('test4.jpg', target_size=(224,224))
img_array = tf.keras.preprocessing.image.img_to_array(img1)
img_array2=tf.keras.applications.inception_v3.preprocess_input(img_array)
img_array2 = tf.expand_dims(img_array2, 0)
predictions = model4.predict(img_array2)
predicted_class = class_subset[np.argmax(predictions[0])]
confidence = round(100 * (np.max(predictions[0])), 2)
print(confidence,predicted_class)

60.49 paper


Update:

In [5]:
IMG_SIZE = 224
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)
base_model1 = InceptionV3(input_shape=IMG_SHAPE, 
                              include_top=False,
                              weights = 'imagenet')

In [6]:
base_model1.trainable=False

In [7]:
model1 = Sequential()
model1.add(base_model1)
model1.add(GlobalAveragePooling2D())
model1.add(Dense(units=7, activation='softmax')) 
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 5, 5, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 7)                 14343     
Total params: 21,817,127
Trainable params: 14,343
Non-trainable params: 21,802,784
_________________________________________________________________


In [8]:
base_model1.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [9]:
model1.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model_details1 = model1.fit(x=train_batches, validation_data=valid_batches, epochs=6, verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 90 steps, validate for 10 steps
Epoch 1/6
90/90 [==============================] - 445s 5s/step - loss: 1.9257 - accuracy: 0.1929 - val_loss: 1.8786 - val_accuracy: 0.2962
Epoch 2/6
90/90 [==============================] - 429s 5s/step - loss: 1.6639 - accuracy: 0.3612 - val_loss: 1.5694 - val_accuracy: 0.3981
Epoch 3/6
90/90 [==============================] - 442s 5s/step - loss: 1.4700 - accuracy: 0.4988 - val_loss: 1.3746 - val_accuracy: 0.4745
Epoch 4/6
90/90 [==============================] - 443s 5s/step - loss: 1.3152 - accuracy: 0.5919 - val_loss: 1.2330 - val_accuracy: 0.5159
Epoch 5/6
90/90 [==============================] - 440s 5s/step - loss: 1.2016 - accuracy: 0.6412 - val_loss: 1.2163 - val_accuracy: 0.5318
Epoch 6/6
90/90 [==============================] - 444s 5s/step - loss: 1.1102 - accuracy: 0.6731 - val_loss: 1.1242 - val_accuracy: 0.5637


In [11]:
model_details1 = model1.fit(x=train_batches, validation_data=valid_batches, epochs=5, verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 90 steps, validate for 10 steps
Epoch 1/5
90/90 [==============================] - 446s 5s/step - loss: 1.0403 - accuracy: 0.6839 - val_loss: 1.0531 - val_accuracy: 0.5987
Epoch 2/5
90/90 [==============================] - 438s 5s/step - loss: 0.9860 - accuracy: 0.7035 - val_loss: 1.0613 - val_accuracy: 0.6051
Epoch 3/5
90/90 [==============================] - 432s 5s/step - loss: 0.9346 - accuracy: 0.7168 - val_loss: 0.9984 - val_accuracy: 0.6433
Epoch 4/5
90/90 [==============================] - 436s 5s/step - loss: 0.8934 - accuracy: 0.7350 - val_loss: 1.0034 - val_accuracy: 0.6401
Epoch 5/5
90/90 [==============================] - 388s 4s/step - loss: 0.8590 - accuracy: 0.7389 - val_loss: 1.0226 - val_accuracy: 0.6401


In [12]:
count=0
for layer in base_model1.layers:
    count+=1
    print(layer,layer.trainable,count)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x0000018959ABD688> False 1
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000018959AD0788> False 2
<tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x0000018959AD9688> False 3
<tensorflow.python.keras.layers.core.Activation object at 0x0000018959AD9908> False 4
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000018959ADA8C8> False 5
<tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x000001895AD17B08> False 6
<tensorflow.python.keras.layers.core.Activation object at 0x0000018959AAF448> False 7
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001895AD1C208> False 8
<tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x000001895AD74808> False 9
<tensorflow.python.keras.layers.core.Activation object at 0x000001895AD7BA08> False 10
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x0

In [16]:
tuning_layer_name = 'conv2d_87'
tuning_layer = base_model1.get_layer(tuning_layer_name)
tuning_index = base_model1.layers.index(tuning_layer)
base_model1.trainable=True
for layer in base_model1.layers[:tuning_index]:
    layer.trainable =  False

In [17]:
for layer in base_model1.layers:
    print(layer,layer.trainable)

<tensorflow.python.keras.engine.input_layer.InputLayer object at 0x0000018959ABD688> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000018959AD0788> False
<tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x0000018959AD9688> False
<tensorflow.python.keras.layers.core.Activation object at 0x0000018959AD9908> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000018959ADA8C8> False
<tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x000001895AD17B08> False
<tensorflow.python.keras.layers.core.Activation object at 0x0000018959AAF448> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x000001895AD1C208> False
<tensorflow.python.keras.layers.normalization.BatchNormalization object at 0x000001895AD74808> False
<tensorflow.python.keras.layers.core.Activation object at 0x000001895AD7BA08> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x000001895AD80788> Fals

In [18]:
model1.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [19]:
model_details1 = model1.fit(x=train_batches, validation_data=valid_batches, epochs=5, verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 90 steps, validate for 10 steps
Epoch 1/5
90/90 [==============================] - 346s 4s/step - loss: 0.7924 - accuracy: 0.7480 - val_loss: 0.9252 - val_accuracy: 0.6879
Epoch 2/5
90/90 [==============================] - 342s 4s/step - loss: 0.6980 - accuracy: 0.7861 - val_loss: 0.9098 - val_accuracy: 0.7006
Epoch 3/5
90/90 [==============================] - 341s 4s/step - loss: 0.6340 - accuracy: 0.8099 - val_loss: 0.8680 - val_accuracy: 0.7070
Epoch 4/5
90/90 [==============================] - 339s 4s/step - loss: 0.5927 - accuracy: 0.8162 - val_loss: 0.8578 - val_accuracy: 0.7102
Epoch 5/5
90/90 [==============================] - 352s 4s/step - loss: 0.5492 - accuracy: 0.8299 - val_loss: 0.8391 - val_accuracy: 0.7070


In [20]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [26]:
img1 = load_img('test3.jpg', target_size=(224,224))
img_array = tf.keras.preprocessing.image.img_to_array(img1)
img_array2=tf.keras.applications.inception_v3.preprocess_input(img_array)
img_array2 = tf.expand_dims(img_array2, 0)
predictions = model1.predict(img_array2)
predicted_class = class_subset[np.argmax(predictions[0])]
confidence = round(100 * (np.max(predictions[0])), 2)
print(confidence,predicted_class)

96.36 plastic


In [27]:
model_details1 = model1.fit(x=train_batches, validation_data=valid_batches, epochs=4, verbose=1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 90 steps, validate for 10 steps
Epoch 1/4
90/90 [==============================] - 343s 4s/step - loss: 0.5255 - accuracy: 0.8449 - val_loss: 0.8128 - val_accuracy: 0.7229
Epoch 2/4
90/90 [==============================] - 345s 4s/step - loss: 0.4910 - accuracy: 0.8586 - val_loss: 0.8067 - val_accuracy: 0.7229
Epoch 3/4
90/90 [==============================] - 342s 4s/step - loss: 0.4719 - accuracy: 0.8582 - val_loss: 0.7866 - val_accuracy: 0.7293
Epoch 4/4
90/90 [==============================] - 340s 4s/step - loss: 0.4397 - accuracy: 0.8694 - val_loss: 0.7877 - val_accuracy: 0.7325
